In [20]:
import tushare as ts
import matplotlib.pyplot as plt
import matplotlib.finance as mpf
import numpy as np
import talib as talib
import pandas as pd
import datetime as dt

#取所有2n天内的低点
def get_low_point(df,columns,n):
    conds = (df[columns]==df[columns])
    for i in range(0,n):
        conds = conds & (df[columns].shift(i+1)>=df[columns]) &  (df[columns].shift(-i-1)>=df[columns])  
    return df[conds] 
#取所有2n天内的高点
def get_height_point(df,col,n):
    conds = (df[columns]==df[columns])
    for i in range(0,n):
        conds = conds & (df[columns].shift(i+1)<=df[columns]) &  (df[columns].shift(-i-1)<=df[columns])  
    return df[conds]

data = ts.get_k_data('002023', ktype='m',autype="qfq",index=False,start='2001-01-01', end='2018-05-15')
#RSI
data["rsi"] = talib.RSI(data['close'].values, timeperiod=9)
data

<urlopen error timed out>


,date,open,close,high,low,volume,code,rsi
0,2004-07-30,2.024,2.120,2.168,1.910,360892.53,002023,NaN
1,2004-08-31,2.120,1.875,2.167,1.723,177060.04,002023,NaN
2,2004-09-30,1.850,2.387,2.495,1.796,158067.84,002023,NaN
3,2004-10-29,2.430,2.592,2.791,2.241,177149.12,002023,NaN
4,2004-11-30,2.567,2.521,2.748,2.340,139323.08,002023,NaN
5,2004-12-31,2.527,2.357,2.713,2.289,133215.68,002023,NaN
6,2005-01-31,2.330,2.081,2.410,1.963,56365.04,002023,NaN
7,2005-02-28,2.001,2.280,2.408,2.001,39928.37,002023,NaN
8,2005-03-31,2.284,2.180,2.494,2.138,117872.96,002023,NaN
9,2005-04-29,2.188,2.147,2.542,2.087,159833.08,002023,50.747922


In [27]:
low=data["low"]
low_Low= low[  (low<=low.shift(2))& (low<=low.shift(1)) & (low<=low.shift(-1)) & (low<=low.shift(-2))  ] 
k_LowPoint= get_low_point(data,"low",2).dropna()
#画出低点
#plt.plot(low_Low.index.values,low_Low.values,'o',color='red')
k_LowPoint 
low_Low


6       1.963
13      1.665
20      1.563
25      1.997
28      1.835
35      2.616
39      2.500
52      1.082
70      4.866
78      6.096
83      6.652
90      3.419
96      3.749
101     3.744
107     4.826
125    10.374
134    13.345
140    11.831
150    12.355
154     9.286
163     7.700
Name: low, dtype: float64